In [22]:
import pandas as pd
import glob
import os
import json
import numpy as np
import datetime
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Storm_Snow_20190120'

In [23]:
actual_weather_folder="/home/jian/Projects/Big_Lots/Weather/Json_data/daily/"
forecast_weather_folder="/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/"

actual_list=[x for x in glob.glob(actual_weather_folder+"*.json") if str(datetime.datetime.now().date()-datetime.timedelta(days=1)) in x]
forecast_list=[x for x in glob.glob(actual_weather_folder+"*.json") if str(datetime.datetime.now().date()-datetime.timedelta(days=1)) in x]

In [24]:
store_open_last_week=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-01-12.csv",dtype=str)
store_open_last_week=store_open_last_week[store_open_last_week['week_end_date']=="2019-01-12"]
store_open_last_week['sales']=store_open_last_week['sales'].astype(float)
store_open_last_week=store_open_last_week[store_open_last_week['sales']>0]
store_open_last_week=store_open_last_week[store_open_last_week['location_id']!="6990"]

len(store_open_last_week['location_id'].unique())

1401

In [25]:
store_list_info=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
store_list_info=store_list_info[store_list_info['location_id'].isin(store_open_last_week['location_id'].tolist())]
store_list_info=store_list_info[['location_id','zip_cd']]
store_list_info['zip_cd']=store_list_info['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_info.head(2)

,location_id,zip_cd
0,3,30906
1,30,43402


In [26]:
empty_list=[]
main_list=[]
i_counters=0
for json_actual in actual_list:
    record_time=json_actual.split("/")[len(json_actual.split("/"))-1].split(".")[0]
    respons=json.load(open(json_actual,"r"))
    df_1_time_point=pd.DataFrame()
    for zip_cd in store_list_info['zip_cd'].unique().tolist():
        try:
            data_for_a_zip=respons[zip_cd]
            weather_main=data_for_a_zip['weather']
            zip_weather_list=[]
            for j in range(len(weather_main)):
                zip_weather_list=zip_weather_list+[weather_main[j]['main']]
            
            df=pd.DataFrame({"zip_cd":zip_cd,"Actual_"+record_time:[zip_weather_list]},index=[i_counters])
            i_counters+=1
            df_1_time_point=df_1_time_point.append(df)
        except:
            empty_list=empty_list+[zip_cd]
            df=pd.DataFrame({"zip_cd":zip_cd,"Actual_"+record_time:['Not_Recorded']},index=[i_counters])
            i_counters+=1
            df_1_time_point=df_1_time_point.append(df)
            
            
    store_list_info=pd.merge(store_list_info,df_1_time_point,on="zip_cd",how="left")


In [27]:
latest_pred_json=json.load(open('/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/2019-01-20: 17.json',"r"))

forecast_df=pd.DataFrame()
i_counter=0
empty_list=[]
df_forecast=pd.DataFrame()
for zip_cd in store_list_info['zip_cd'].unique().tolist():
    try:
        data_for_a_zip=latest_pred_json[zip_cd]
        df=pd.DataFrame({"zip_cd":zip_cd},index=[i_counter])
        i_counters+=1
        
        for i in range(40):
            time_point_data=data_for_a_zip['list'][i]
            dt_txt=datetime.datetime.utcfromtimestamp(time_point_data['dt']-60*60*5).strftime("%Y-%m-%d %H:%M:%S")
            weather_main=time_point_data['weather']
            zip_weather_list=[]
            for j in range(len(weather_main)):
                zip_weather_list=zip_weather_list+[weather_main[j]['main']]
                df["Predict_"+dt_txt]=[zip_weather_list]
        df_forecast=df_forecast.append(df)
    except:
        empty_list=empty_list+[zip_cd]
        df=pd.DataFrame({"zip_cd":zip_cd},index=[i_counter])
        i_counters+=1
        df_forecast=df_forecast.append(df)

In [28]:
store_list_info=pd.merge(store_list_info,df_forecast,on="zip_cd",how="left")

In [29]:
store_list_info['Snow_Sunday_Monday']=store_list_info['Actual_2019-01-20: 11']+store_list_info['Actual_2019-01-20: 14']+store_list_info['Actual_2019-01-20: 17']+\
                                        store_list_info['Predict_2019-01-20 19:00:00']+store_list_info['Predict_2019-01-20 22:00:00']+\
                                        store_list_info['Predict_2019-01-21 01:00:00']+store_list_info['Predict_2019-01-21 04:00:00']+\
                                        store_list_info['Predict_2019-01-21 07:00:00']+store_list_info['Predict_2019-01-21 10:00:00']+\
                                        store_list_info['Predict_2019-01-21 13:00:00']+store_list_info['Predict_2019-01-21 16:00:00']+\
                                        store_list_info['Predict_2019-01-21 19:00:00']+store_list_info['Predict_2019-01-21 22:00:00']
store_list_info['Snow_Sunday_Monday']=store_list_info['Snow_Sunday_Monday'].fillna("Not_Recorded")                    
store_list_info['Snow_Sunday_Monday']=store_list_info['Snow_Sunday_Monday'].apply(lambda x: set(x))

store_list_Snow_SundayMonday=store_list_info[store_list_info['Snow_Sunday_Monday'].apply(lambda x: "Snow" in x)]

In [30]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)

store_level_zips=store_level_zips[['location_id','revenue_flag','zip']].drop_duplicates()

store_level_zips=store_level_zips[store_level_zips['location_id'].isin(store_open_last_week['location_id'].unique().tolist())]

df_impacted=store_level_zips[store_level_zips['location_id'].isin(store_list_Snow_SundayMonday['location_id'])]
df_Not_impacted=store_level_zips[~store_level_zips['location_id'].isin(store_list_Snow_SundayMonday['location_id'])]

df_impacted_P_S=df_impacted[df_impacted['revenue_flag'].isin(["P","S"])]
df_Not_impacted_P_S=df_Not_impacted[df_Not_impacted['revenue_flag'].isin(["P","S"])]

df_impacted_P_S_Not_Others_PS=df_impacted_P_S[~df_impacted_P_S['zip'].isin(df_Not_impacted_P_S["zip"].unique().tolist())]
'''
df_impacted_T=df_impacted[df_impacted['revenue_flag']=="T"]
df_impacted_T_Not_Others_T=df_impacted_T[~df_impacted_T['zip'].isin(df_Not_impacted["zip"].unique().tolist())]

df_PST=df_Not_impacted_P_S.append(df_impacted_T_Not_Others_T)
'''
# No need to change in this way because the whole TA mostly consider P/S zips

'\ndf_impacted_T=df_impacted[df_impacted[\'revenue_flag\']=="T"]\ndf_impacted_T_Not_Others_T=df_impacted_T[~df_impacted_T[\'zip\'].isin(df_Not_impacted["zip"].unique().tolist())]\n\ndf_PST=df_Not_impacted_P_S.append(df_impacted_T_Not_Others_T)\n'

# Add 10 miles zips

In [31]:
store_list_org=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")


new_store_list=[x for x in store_list_Snow_SundayMonday['location_id'].tolist() if x not in df_impacted_P_S_Not_Others_PS['location_id'].unique().tolist()]
len(new_store_list)

new_store_list_lat_lng=store_list_org[store_list_org['location_id'].isin(new_store_list)][['location_id','latitude_meas','longitude_meas']]
new_store_list_lat_lng['latitude_meas']=new_store_list_lat_lng['latitude_meas'].astype(float)
new_store_list_lat_lng['longitude_meas']=new_store_list_lat_lng['longitude_meas'].astype(float)
new_store_list_lat_lng=new_store_list_lat_lng.reset_index()
del new_store_list_lat_lng['index']

In [32]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
from haversine import haversine
all_new_store_zips_df=pd.DataFrame()
for i in range(len(new_store_list_lat_lng)):
    location=new_store_list_lat_lng['location_id'][i]
    store_center=[new_store_list_lat_lng['latitude_meas'][i],new_store_list_lat_lng['longitude_meas'][i]]
    store_df=pd.DataFrame()
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location,"zip":zip_cd,"miles":dist},index=[0])
            store_df=store_df.append(df)
            
    all_new_store_zips_df=all_new_store_zips_df.append(store_df)
    
print(all_new_store_zips_df.shape)
all_new_store_zips_df_not_in_Others_PS=all_new_store_zips_df[~all_new_store_zips_df['zip'].isin(df_Not_impacted_P_S["zip"].unique().tolist())]
print(all_new_store_zips_df_not_in_Others_PS.shape)

(586, 3)
(523, 3)


In [33]:
df_zip_exclude_unque=df_impacted_P_S_Not_Others_PS[['zip']].append(all_new_store_zips_df_not_in_Others_PS[['zip']]).drop_duplicates()

del store_list_org['zip_cd']
store_list_info=pd.merge(store_list_info,store_list_org,on="location_id",how="left")

In [34]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Storm_Snow_20190120/BL_Zips_for_Snow_20190120-21_PS_newstore_10_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_zip_exclude_unque.to_excel(writer,"Zips_Influced_to_Suppress",index=False)

store_list_Snow_SundayMonday=store_list_Snow_SundayMonday[['location_id','zip_cd','Snow_Sunday_Monday']+[x for x in store_list_Snow_SundayMonday.columns.tolist() if x not in ['location_id','zip_cd','Snow_Sunday_Monday']]]
store_list_Snow_SundayMonday.to_excel(writer,"stores_impacted",index=False)

df_impacted_P_S_Not_Others_PS.to_excel(writer,"zips_by_store_PS",index=False)
all_new_store_zips_df_not_in_Others_PS.to_excel(writer,"zips_new_stores_suppress",index=False)
all_new_store_zips_df.to_excel(writer,"zips_new_stores_all",index=False)
store_list_info.to_excel(writer,"all_open_store_weather",index=False)

writer.save()